In [1]:
!pip install pyjwt

In [3]:
with open("../key.txt", "r") as file:
    API_KEY_CLARIN = file.read().strip()

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"
CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"
GOOGLE_KEY_PATH = "google_api.json"
GOOGLE_OAUTH2_TOKEN_URL = "https://oauth2.googleapis.com/token"

In [1]:
import requests
import pandas as pd
import json
import time
import jwt
import requests

In [4]:
with open(GOOGLE_KEY_PATH) as f:
    google_creds = json.load(f)

In [5]:
def get_access_token():
    issued_at = int(time.time())
    expiration_time = issued_at + 3600

    payload = {
        "iss": google_creds["client_email"],  # Issuer (Service Account Email)
        "scope": "https://www.googleapis.com/auth/cloud-translation",  # Translation API scope
        "aud": GOOGLE_OAUTH2_TOKEN_URL,
        "iat": issued_at,  # Issued At
        "exp": expiration_time  # Expiry
    }

    # Create a JWT signed with the private key
    private_key = google_creds["private_key"]
    jwt_token = jwt.encode(payload, private_key, algorithm="RS256")

    # Exchange JWT for OAuth 2.0 access token
    token_request_data = {
        "grant_type": "urn:ietf:params:oauth:grant-type:jwt-bearer",
        "assertion": jwt_token
    }

    response = requests.post(GOOGLE_OAUTH2_TOKEN_URL, data=token_request_data)
    
    if response.status_code == 200:
        return response.json()["access_token"]
    else:
        raise Exception(f"Error obtaining access token: {response.text}")

def google_translate(text):
    token = get_access_token()
    url = "https://translation.googleapis.com/language/translate/v2"

    headers = {
        "Authorization": f"Bearer {token}",
        "Content-Type": "application/json"
    }
    payload = {
        "q": text,
        "target": "pl"
    }
    response = requests.post(url, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json()["data"]["translations"][0]["translatedText"]
    else:
        return f"Error: {response.text}"


In [4]:
def get_models_clarin():
    headers = {
        'Authorization': f'Bearer {API_KEY_CLARIN}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json() 

In [7]:
def prompt_chat(model_id, prompt):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected

In [8]:
def prompt_chat_custom_temperature(model_id, prompt, temperature = 0.3):
    url = COMPLETIONS_ENDPOINT
    headers = {"Authorization": f"Bearer {API_KEY_CLARIN}", "Content-Type": "application/json"}
    messages = []
    messages.append({"role": "user", "content": prompt})
    data = {
        "model": model_id,
        "messages": messages,
        "temperature": temperature
        #"max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        try:
            data = response.json()
            if isinstance(data, dict) and "choices" in data and data["choices"]:
                return data["choices"][0].get("message", {}).get("content", None)
            else:
                return None  # Handle unexpected response structure
        except (ValueError, TypeError, AttributeError):
            return None  # Handle cases where response.json() is None or not as expected

In [5]:
models = get_models_clarin()
print(models)

{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct', 'type': 'chat'}, {'id': 'c4ai-command-a', 'full_name': 'CohereForAI/c4ai-command-a-03-2025', 'name': 'CohereForAI/c4ai-command-a-03-2025', 'type': 'chat'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus', 'type': 'chat'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'type': 'chat'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B', 'type': 'chat'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct', 'type': 'chat'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210', 'type': 'chat'}, {'id': 'dariah-pllum12b-18_04-newformat', 'full_name': 'dariah/pllum12b-18_04-new

In [10]:
df = pd.read_csv("data/dataset.csv", delimiter="|")
df

,en_text,pl_text
0,"Hey, everyone! Wanna hear about Fyresdal?","Ej, ludzie! Chcecie posłuchać o Fyresdal?"
1,"Hey, good folk! Wanna hear what our brave guar...","Ej, ludzie! Chcecie usłyszeć, co nasz dzielny ..."
2,"Shut your trap, dammit!","Stul pysk, do cholery!"
3,Four against one - and you came out alive.,Czterech na jednego. A dałeś radę.
4,Shame to let such a lad waste away behind bars...,"Szkoda, żeby taki chłop marniał za kratkami......"
...,...,...
8375,Farewell.,Bywaj.
8376,Farewell.,Bywaj.
8377,What do you have on offer?,Pokaż mi swoje towary.
8378,Show me what you have in stock.,Pokaż mi swoje towary.


In [13]:
models = ["bielik", "mixtral-8x22B", "llama3.1", "google"]

In [ ]:
sampled_df = df.sample(n=1000, random_state=28, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]
    polish_text = row["pl_text"]

    for model in models:
        if model == "google":
            translation = google_translate(english_text)
        else:
            translation = prompt_chat(model, f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'")
        
        sampled_df.at[index, f'{model}_translation'] = translation

sampled_df.to_csv("results/translations.csv", sep='|', index=False)

In [ ]:
sampled_df

,en_text,pl_text,bielik_translation,mixtral-8x22B_translation,llama3.1_translation,google_translation
0,Fire!,Gorze!,Pożar!,Ogień!,Ognia!,Ogień!
1,Find me a king who'll kill a god and plough a ...,"Posłucham tylko tego króla, który zabije boga ...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i podora b...",'Znajdź mi króla który zabił boga i zaora bogi...,"Znajdź mi króla, który zabije boga i zrujnuje ..."
2,Britt wasn't lying.,A więc Britt nie kłamała.,Britt nie kłamał.,Britt nie kłamała.,Britt nie kłamał.,Britt nie kłamał.
3,"You don't believe in the gods, Lugos.","Ty nie wierzysz w bogów, Lugos.","""Nie wierzysz w bogów, Lugos.""","Nie wierzysz w bogów, Lugosie.","Nie wierzysz bogom, Lugos.","Nie wierzysz w bogów, Lugosie."
4,"Where, exactly?",A skąd?,Gdzie dokładnie?,Dokładnie gdzie?,Gdzie dokładnie?,Gdzie dokładnie?
...,...,...,...,...,...,...
995,Because… no bite marks on it… Doesn't seem str...,Bo nie ma na nim śladów żerowania. Nie wydaje ...,Ponieważ... nie ma na nim śladów ugryzień... N...,Ponieważ… nie ma na nie śladów ukąszeń… Nie w...,'Ponieważ...brak śladów ugryzień... Nie jest c...,Ponieważ... nie ma na nim śladów ugryzień... N...
996,Not about to help you with this.,Nie pomogę ci.,Nie mam zamiaru pomagać ci w tym.,Nie zamierzam ci pomagać w tym.,Nie zamierzam ci w tym pomóc.,Nie zamierzam ci w tym pomóc.
997,Try to remember. This is important to us.,Spróbuj sobie przypomnieć. To dla nas ważne...,Spróbuj zapamiętać. To jest dla nas ważne.,Spróbuj pamiętać. To jest dla nas ważne.,Staraj się pamiętać. Jest to dla nas ważne.,Spróbuj zapamiętać. To jest dla nas ważne.
998,I understand… But I've told you all I know.,"Rozumiem. Ale nie wiem nic więcej ponad to, co...","Rozumiem... Ale powiedziałem ci wszystko, co w...",Rozumiem... Ale powiedziałem/powiedziałam już...,"Rozumiem... Ale powiedziałem Ci już wszystko, ...","Rozumiem… Ale powiedziałem ci wszystko, co wiem."


In [25]:
sampled_df2 = df.sample(n=100, random_state=28, ignore_index=True)
temperatures = [round(i, 1) for i in [x * 0.1 for x in range(11)]]

for index, row in sampled_df2.iterrows():
    english_text = row["en_text"]
    
    if "google" in models:
        google_translation = google_translate(english_text)
        sampled_df2.at[index, 'google_translation'] = google_translation
    
    for temperature in temperatures:
        for model in models:
            if model != "google":
                translation = prompt_chat_custom_temperature(
                    model, 
                    f"Przetłumacz na polski i nie dodawaj żadnych dodatkowych wyjaśnień, napisz tylko samo tłumaczenie: '{english_text}'", 
                    temperature
                )
                sampled_df2.at[index, f'{model}_translation_{temperature}'] = translation
    
    if (index + 1) % 10 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df2)} samples...")

sampled_df2.to_csv("results/translations_temperatures.csv", sep='|', index=False)

Processed 10 out of 100 samples...
Processed 20 out of 100 samples...
Processed 30 out of 100 samples...
Processed 40 out of 100 samples...
Processed 50 out of 100 samples...
Processed 60 out of 100 samples...
Processed 70 out of 100 samples...
Processed 80 out of 100 samples...
Processed 90 out of 100 samples...
Processed 100 out of 100 samples...


In [26]:
sampled_df2

,en_text,pl_text,google_translation,bielik_translation_0.0,mixtral-8x22B_translation_0.0,llama3.1_translation_0.0,bielik_translation_0.1,mixtral-8x22B_translation_0.1,llama3.1_translation_0.1,bielik_translation_0.2,...,llama3.1_translation_0.7,bielik_translation_0.8,mixtral-8x22B_translation_0.8,llama3.1_translation_0.8,bielik_translation_0.9,mixtral-8x22B_translation_0.9,llama3.1_translation_0.9,bielik_translation_1.0,mixtral-8x22B_translation_1.0,llama3.1_translation_1.0
0,Fire!,Gorze!,Ogień!,Pożar!,Ogień!,Ogień!,Pożar!,Ogień!,Ogień!,Pożar!,...,Ogień!,Pożar!,Ogień!,Ogień!,Pożar!,Ogień!,Ogień!,Pożar!,Ogień!,Ogień!
1,Find me a king who'll kill a god and plough a ...,"Posłucham tylko tego króla, który zabije boga ...","Znajdź mi króla, który zabije boga i zrujnuje ...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i zaorze b...","Znajdź mi króla, który zabiłby boga i zaorał b...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i zaorze b...","Znajdź mi króla, który zabiłby boga i zaorał b...","Znajdź mi króla, który zabije boga i zaorze bo...",...,"Znajdź mi króla, który zabiłboga i zaora bogin...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i orze bog...","Znajdź mi króla, który zabiłboga i zaora bogin...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i zorze zi...","Znajdź mi króla, który zabiłboga i zaora bogin...","Znajdź mi króla, który zabije boga i zaorze bo...","Znajdź mi króla, który zabije boga i zorze zi...","'Znajdź mi króla, który zabije boga i zaora bo..."
2,Britt wasn't lying.,A więc Britt nie kłamała.,Britt nie kłamał.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,...,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt się nie kłamała.,Britt nie kłamała.,Britt nie kłamała.,Britt nie kłamał.
3,"You don't believe in the gods, Lugos.","Ty nie wierzysz w bogów, Lugos.","Nie wierzysz w bogów, Lugosie.","""Nie wierzysz w bogów, Lugos.""","Nie wierzysz w bogów, Lugosie.","Nie wierzysz w bogów, Lugos.","""Nie wierzysz w bogów, Lugos.""","Nie wierzysz w bogów, Lugosie.","Nie wierzysz w bogów, Lugos.","""Nie wierzysz w bogów, Lugos.""",...,"Nie wierzysz w bogów, Lugosie.","""Nie wierzysz w bogów, Lugos.""","Nie wierysz w bogów, Lugosie.","Nie wierzysz w bogów, Lugo.","Nie wierzysz w bogów, Luatos.","Nie wierzysz w bogów, Lugosie.","Nie wierzysz w bogów, Lugos.","""Nie wierzysz w bogów, Lugos.""","Nie wierzysz w bogów, Lugosie.","Nie wierzysz w bogów, Lugos."
4,"Where, exactly?",A skąd?,Gdzie dokładnie?,Gdzie dokładnie?,Dokładnie gdzie?,Gdzie dokładnie?,Gdzie dokładnie?,Dokładnie gdzie?,Gdzie dokładnie?,Gdzie dokładnie?,...,Gdzie dokładnie?,Gdzie dokładnie?,Dokładnie gdzie?,Gdzie dokładnie?,Gdzie dokładnie?,"Dokładnie, gdzie?",Gdzie dokładnie?,Gdzie dokładnie?,"Dokładnie, gdzie?",gdzie dokładnie?
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,Feeble. But I expected as much.,Słabiutko. Ale spodziewałem się tego.,Słabe. Ale spodziewałem się tego.,Słaby. Ale tego się spodziewałem.,Słabe. Ale to było oczywiste.,Słaby. Ale spodziewałem się czegoś takiego.,Słaby. Ale tego się spodziewałem.,Słabe. Ale to było oczywiste.,Słaby. Ale spodziewałem się czegoś takiego.,Słaby. Ale tego się spodziewałem.,...,Słaby. Ale spodziewałem się czegoś takiego.,Słaby. Ale tego się spodziewałem.,Słabe. Ale to mi się przecież spodziewałem.,Słaby. Ale spodziewałem się czegoś takiego.,Słaby. Ale tego się spodziewałem.,Słaby. Ale tak właśnie spodziewałem się.,Słaby. Ale tak się spodziewałem.,Słaby. Ale tego się spodziewałem.,'Słaby. Ale byłam gotowa na to.' lub 'Słaby. ...,Słaby. Ale tak się spodziewałem.
96,It shan't take long.,To nie zajmie długo.,To nie potrwa długo.,Nie zajmie to długo.,N

In [ ]:
sampled_df = df.sample(n=1000, random_state=28, ignore_index=True)

for index, row in sampled_df.iterrows():
    english_text = row["en_text"]
    polish_text = row["pl_text"]

    for model in models:
        if model == "google":
            translation = google_translate(english_text)
        else:
            translation = prompt_chat(model, f"Translate to polish and don't add any explanations, write only translation: '{english_text}'")
        
        sampled_df.at[index, f'{model}_translation'] = translation

sampled_df.to_csv("results/translations_en_prompt.csv", sep='|', index=False)

: 

In [14]:
models = ["bielik", "mixtral-8x22B", "llama3.1"]

In [15]:
sampled_df2 = df.sample(n=100, random_state=28, ignore_index=True)
temperatures = [round(i, 1) for i in [x * 0.1 for x in range(11)]]

for index, row in sampled_df2.iterrows():
    english_text = row["en_text"]
    
    for temperature in temperatures:
        for model in models:
            if model != "google":
                translation = prompt_chat_custom_temperature(
                    model, 
                    f"Translate to polish and don't add any explanations, write only translation: '{english_text}'", 
                    temperature
                )
                sampled_df2.at[index, f'{model}_translation_{temperature}'] = translation
    
    if (index + 1) % 10 == 0:
        print(f"Processed {index + 1} out of {len(sampled_df2)} samples...")

sampled_df2.to_csv("results/translations_temperatures_en_prompt.csv", sep='|', index=False)

Processed 10 out of 100 samples...
Processed 20 out of 100 samples...
Processed 30 out of 100 samples...
Processed 40 out of 100 samples...
Processed 50 out of 100 samples...
Processed 60 out of 100 samples...
Processed 70 out of 100 samples...
Processed 80 out of 100 samples...
Processed 90 out of 100 samples...
Processed 100 out of 100 samples...
